In [1]:
import os
import librosa
import librosa.display
import warnings
warnings.filterwarnings('ignore')
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv

In [2]:
# 파일 경로
# audio_path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download/classic/converted"
audio_path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download/converted/2022-03-21_"

In [3]:
def song_names(audio_path):                      # audio_path : 음원 파일들의 폴더 경로
    files = list(os.listdir(f'{audio_path}'))

    songs = []
    paths = []

    for file in files:
        song = f'{file}'
        path = f'{audio_path}/{file}'
        songs.append(song)
        paths.append(path)

    return songs, paths

In [4]:
def soundwave(paths):
    audio_files = []
    song_name = []

    for path in paths:
        try:
            p = path.split("/")[-1]
            y, sr = librosa.load(path, duration = 30)
            y, _ = librosa.effects.trim(y)
            song_name.append(p)
            audio_files.append(y)

        except ValueError as val:
            print(path, val)

    return audio_files, song_name, sr # sr 디폴트 : 22050 / 변경 가능한 sr 수치 : 44100

In [5]:
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y)
    features['sample_silence'] = len(y) - len(y_sound)

    # Using librosa to calculate the features
    features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
    features['rmse'] = librosa.feature.rms(y).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
    # harmony, perceptral treatment (+@ 부분)
    harm, perc = librosa.effects.hpss(y)
    features['harm'] = harm.ravel()
    features['perc'] = perc.ravel()

    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, sr=sr)   # n_mfcc=20 (default) 
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
#    onset_env = librosa.onset.onset_strength(y=audio_file, sr=sr)
#    dict_agg_features['tempo'] = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [6]:
def final_extraction(dataset_dir=audio_path):
#    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    last_features=[]

    songs, paths = song_names(audio_path)
    audio_files, song_name, sr = soundwave(paths)

    for audio in audio_files:
        features = get_features(audio, sr)
        last_features.append(features)

    return last_features, songs

In [ ]:
%%time

last_features, songs = final_extraction()

In [ ]:
def df_all_features(last_features, songs):
    df_features = pd.DataFrame(last_features)
    df_songs = pd.DataFrame(songs, columns=['song'])

    return pd.concat([df_songs, df_features], axis=1)

In [ ]:
df_all_features = df_all_features(last_features, songs)
df_all_features.head()

In [ ]:
# export the data to a csv file
df_all_features.to_csv('/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/feature_result.csv')

In [ ]:
# import the data
df_all_features = pd.read_csv('/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/feature_result.csv', index_col = 0)

In [ ]:
df_all_features

### 시도해봐야 하는 것
- 에러 로그 찍기
- df에 곡명 컬럼 추가하기

In [ ]:
# list(df_features.columns)